# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) abstracts key Python concepts into a streamlined format, facilitating a "minimalist" code layer for constructing chains of LangChain components. LCEL offers robust support for:

1. Rapid development of chains.
2. Advanced features like streaming, asynchronous processing, parallel execution, and more.
3. Seamless integration with LangSmith and LangServe.

In [1]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama
from rich import print as pp

# Create a prompt template for generating book summaries
prompt = ChatPromptTemplate.from_template(
    "Summarize the book titled '{book_title}' in three sentences."
)

# Use a specific LLM model
model = ChatOllama(model="llama3.2")


## LCEL Syntax

In [2]:

output_parser = StrOutputParser()

from langchain.chains import LLMChain

# Create a chain for summarizing books
chain = LLMChain(
    prompt=prompt,
    llm=model,
    output_parser=output_parser
)


/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_58803/2803915055.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [3]:

# Run the chain
out = chain.run(book_title="1984 by George Orwell")
print(out)


/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_58803/355639390.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  out = chain.run(book_title="1984 by George Orwell")


Here is a summary of "1984" by George Orwell:

In a dystopian future, the totalitarian government of Oceania, led by a figure known as Big Brother, exercises total control over its citizens through constant surveillance and propaganda. The protagonist, Winston Smith, begins to rebel against the government's all-pervasive influence, starting an illicit love affair with a fellow worker, Julia, and secretly writing in a forbidden diary. Ultimately, Winston's desire for freedom and individuality is crushed by the authorities, who capture him and subject him to a brutal process of physical and psychological torture, erasing his very existence from history.


In [4]:

# Alternative LCEL-style chain
lcel_chain = prompt | model | output_parser

# Run the chain
out = lcel_chain.invoke({"book_title": "To Kill a Mockingbird"})
print(out)


Here is a summary of "To Kill a Mockingbird" in three sentences:

Set in the Deep South during the 1930s, the novel follows the experiences of Scout Finch and her family as they navigate issues of racism, injustice, and childhood innocence in the small town of Maycomb, Alabama. The story centers around the defense of Tom Robinson, a black man falsely accused of raping a white woman, as Scout's father, Atticus Finch, agrees to take on the case despite knowing he'll face prejudice and hostility. Through Atticus's courageous yet principled defense of Tom, the novel explores themes of empathy, compassion, and understanding, ultimately leading to a poignant exploration of the complexities of human nature and the importance of doing what is right in the face of overwhelming opposition.


## How the Pipe Operator Works
To truly grasp LCEL, let's examine how the pipe operation functions. It takes output from the **right** and feeds it to the **left**—but since this isn't standard Python, how is it implemented? We can create our own version using simple functions.

We'll utilize the `__or__` method in Python class objects. When we combine two classes like `chain = class_a | class_b`, the Python interpreter checks for the presence of the `__or__` method in these classes. If it exists, the expression `|` is translated to `chain = class_a.__or__(class_b)`. 

This means both of the following patterns yield the same result:

```python
# Object approach
chain = class_a.__or__(class_b)
chain("some input")

# Pipe approach
chain = class_a | class_b
chain("some input")
```

With this understanding, we can create a `Runnable` class that takes a function and transforms it into a chainable function using the pipe operator `|`.



In [5]:

class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)


def subtract_three(x):
    return x - 3


def divide_by_two(x):
    return x / 2


subtract_three = Runnable(subtract_three)
divide_by_two = Runnable(divide_by_two)

chain = subtract_three | divide_by_two
print(chain(10))


3.5


## LCEL Deep Dive
Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

In [6]:

from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Create an embedding model
embedding = OllamaEmbeddings(model="snowflake-arctic-embed:33m") 
# Create vector stores with different content
vecstore_a = InMemoryVectorStore.from_texts(
    ["Python is a programming language.", "It is popular for data science."],
    embedding=embedding
)
vecstore_b = InMemoryVectorStore.from_texts(
    ["Python supports object-oriented programming.", "It is used for web development."],
    embedding=embedding
)


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [7]:

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Provide details about Python:

Context: {context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(prompt_str)

retriever = RunnableParallel(
    {'context': retriever_a, 'question': RunnablePassthrough()},
)

chain = retriever | prompt | model | output_parser

pp(chain)

out = chain.invoke("What are the uses of Python?")
print(out)


RunnableSequence(
    first=RunnableParallel(
        steps__={
            'context': VectorStoreRetriever(
                tags=['InMemoryVectorStore', 'OllamaEmbeddings'],
                vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x11db48c10>,
                search_kwargs={}
            ),
            'question': RunnablePassthrough()
        }
    ),
    middle=[
        ChatPromptTemplate(
            input_variables=['context', 'question'],
            input_types={},
            partial_variables={},
            messages=[
                HumanMessagePromptTemplate(
                    prompt=PromptTemplate(
                        input_variables=['context', 'question'],
                        input_types={},
                        partial_variables={},
                        template='Provide details about Python:\n\nContext: {context}\n\nQuestion: 
{question}\n\nAnswer:'
                    ),
                    additional_kwargs={}
                )
            ]
        ),
        ChatOllama(model='llama3.2')
    ],
    last=StrOutputParser()
)

Python is a versatile and widely-used programming language with a multitude of applications across various domains. Some of the main uses of Python include:

1. **Data Science and Machine Learning**: Python is an essential tool in data science, particularly when it comes to machine learning, deep learning, and natural language processing. Libraries like NumPy, pandas, and scikit-learn provide efficient data structures and algorithms for data analysis, modeling, and visualization.
2. **Web Development**: Python can be used for web development using popular frameworks like Django and Flask. These frameworks enable developers to build scalable, secure, and maintainable web applications quickly.
3. **Automation and Scripting**: Python's easy-to-learn syntax and vast number of libraries make it an ideal choice for automating tasks, such as data processing, file management, and system administration.
4. **Artificial Intelligence and Robotics**: Python is used in AI and robotics to build inte

## Runnable Lambdas
The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [8]:

def calc_cube(x):
    return x ** 3

def calc_reciprocal(x):
    return 1 / x

from langchain_core.runnables import RunnableLambda

calc_cube = RunnableLambda(calc_cube)
calc_reciprocal = RunnableLambda(calc_reciprocal)

chain = calc_cube | calc_reciprocal
pp(chain)

result = chain.invoke(4)
print(result)


RunnableSequence(first=RunnableLambda(calc_cube), middle=[], last=RunnableLambda(calc_reciprocal))

0.015625


In [9]:

def extract_first_sentence(x):
    return x.split(".")[0] + "."

get_first_sentence = RunnableLambda(extract_first_sentence)

prompt_str = "Share an interesting fact about {topic}."
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser | get_first_sentence
pp(chain)

fact = chain.invoke({"topic": "Machine Learning"})
print(fact)


RunnableSequence(
    first=ChatPromptTemplate(
        input_variables=['topic'],
        input_types={},
        partial_variables={},
        messages=[
            HumanMessagePromptTemplate(
                prompt=PromptTemplate(
                    input_variables=['topic'],
                    input_types={},
                    partial_variables={},
                    template='Share an interesting fact about {topic}.'
                ),
                additional_kwargs={}
            )
        ]
    ),
    middle=[ChatOllama(model='llama3.2'), StrOutputParser()],
    last=RunnableLambda(extract_first_sentence)
)

Here's an interesting fact about Machine Learning:

**The "Curse of Dimensionality"**

In the 1960s, mathematician Richard Ashford discovered that as the number of features (or dimensions) in a dataset increases, the amount of data required to achieve accurate predictions also increases exponentially.
